# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
print(data.head())

  id ad  education  ofjobs  yearsexp  honors  volunteer  military  empholes  \
0  b  1          4       2         6       0          0         0         1   
1  b  1          3       3         6       0          1         1         0   
2  b  1          4       1         6       0          0         0         0   
3  b  1          3       4         6       0          1         0         1   
4  b  1          3       3        22       0          0         0         0   

   occupspecific  ...  compreq  orgreq  manuf  transcom  bankreal trade  \
0             17  ...      1.0     0.0    1.0       0.0       0.0   0.0   
1            316  ...      1.0     0.0    1.0       0.0       0.0   0.0   
2             19  ...      1.0     0.0    1.0       0.0       0.0   0.0   
3            313  ...      1.0     0.0    1.0       0.0       0.0   0.0   
4            313  ...      1.0     1.0    0.0       0.0       0.0   0.0   

  busservice othservice  missind  ownership  
0        0.0        0.0     

In [3]:
w = data[data.race=='w']
b = data[data.race=='b']
print('Do w and b have the same number of observations?',len(w)==len(b))

total_callbacks_w = sum(w.call)
total_obs_w = len(w.call)
rate_callbacks_w = total_callbacks_w/total_obs_w

total_callbacks_b = sum(b.call)
total_obs_b = len(b.call)
rate_callbacks_b = total_callbacks_b/total_obs_b

print('\nnumber of callbacks for white-sounding names:',total_callbacks_w)
print('number of observations for white-sounding names:',total_obs_w)
print('rate of callbacks for white-sounding names:',rate_callbacks_w)

print('\nnumber of callbacks for black-sounding names:',total_callbacks_b)
print('number of observations for black-sounding names:',total_obs_b)
print('rate of callbacks for white-sounding names:',rate_callbacks_b)

Do w and b have the same number of observations? True

number of callbacks for white-sounding names: 235.0
number of observations for white-sounding names: 2435
rate of callbacks for white-sounding names: 0.09650924024640657

number of callbacks for black-sounding names: 157.0
number of observations for black-sounding names: 2435
rate of callbacks for white-sounding names: 0.06447638603696099


1-) What test is appropriate for this problem? Does CLT apply?  
I would use bootstrap to test if callbacks for white and black-sounding names come from the same distribution. Also, if the rates of callbacks for white and black-sounding names are the same. CLT should apply since total observations for each group is 2435, much larger than 30.

2-) What are the null and alternate hypotheses?  
First test:  
H0: callbacks for white and black-sounding names come from the same distribution.  
H1: they come from different distributions.  
Second test:  
H0: the rates of callbacks for white and black-sounding names are the same.  
H1: they are different.

In [4]:
## 3-) H0: callbacks for white and black-sounding names come from the same distribution.
##     H1: they come from different distributions.

def permutation_sample(data1, data2):
    data = np.concatenate((data1, data2))
    permuted_data = np.random.permutation(data)
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    perm_replicates = np.empty(size)
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

def rate_of_ones(data):
    return sum(data)/len(data)

def diff_rates_of_ones(data_1, data_2):
    diff = rate_of_ones(data_1) - rate_of_ones(data_2)
    return diff

np.random.seed(10)
empirical_diff_rates = diff_rates_of_ones(w.call, b.call)
perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_rates_of_ones, size=10000)

p = np.sum(perm_replicates >= empirical_diff_rates)/len(perm_replicates)
print('H0: callbacks for w and b-sounding names come from the same distribution.')
print('Procedure using bootstrap and permutation.')
print('p-value =', p)
print('H0 is rejected.')

H0: callbacks for w and b-sounding names come from the same distribution.
Procedure using bootstrap and permutation.
p-value = 0.0
H0 is rejected.


In [5]:
## 3-) H0: the rates of callbacks for white and black-sounding names are the same.
##     H1: they are different.

def bootstrap_replicate_1d(data, func): 
    return func(np.random.choice(data, len(data)))

def draw_bs_reps(data, func, size=1):
    return np.array([bootstrap_replicate_1d(data, func) for _ in range(size)])

def rate_of_ones(data):
    return sum(data)/len(data)

bs_replicates_w = draw_bs_reps(w.call, rate_of_ones, size=10000)
bs_replicates_b = draw_bs_reps(b.call, rate_of_ones, size=10000)
conf_int_95_w = np.percentile(bs_replicates_w, [2.5, 97.5])
conf_int_95_b = np.percentile(bs_replicates_b, [2.5, 97.5])
print('H0: the rates of callbacks for w and b-sounding names are the same.')
print('Procedure using bootstrap and 95% confidence intervals.')
print('95% confidence interval for rate of callbacks (w) =', conf_int_95_w)
print('95% confidence interval for rate of callbacks (b) =', conf_int_95_b)
print('Intervals do not overlap each other.')
print('H0 is rejected.')

## Alternative procedure.
bs_replicates = bs_replicates_w - bs_replicates_b
hypothetical_diff_rates = 0
p = np.sum(bs_replicates <= hypothetical_diff_rates)/len(bs_replicates)
print('\nH0: the rates of callbacks for w and b-sounding names are the same.')
print('Procedure using bootstrap and p-value for difference <= 0.')
print('p-value =', p)
print('H0 is rejected.')

hypothetical_diff_rates = 0.018
p = np.sum(bs_replicates <= hypothetical_diff_rates)/len(bs_replicates)
print('Procedure using bootstrap and p-value for difference <= 0.018.')
print('p-value =', p)
print('H0 is rejected.')

## Checking how big that difference could be.
hypothetical_diff_rates = 0.045
p = np.sum(bs_replicates >= hypothetical_diff_rates)/len(bs_replicates)
print('\nHow big the difference could be?')
print('Procedure using bootstrap and p-value for difference >= 0.045.')
print('p-value =', p)
print('H0 is rejected.')

H0: the rates of callbacks for w and b-sounding names are the same.
Procedure using bootstrap and 95% confidence intervals.
95% confidence interval for rate of callbacks (w) = [0.08501027 0.10841889]
95% confidence interval for rate of callbacks (b) = [0.0550308  0.07433265]
Intervals do not overlap each other.
H0 is rejected.

H0: the rates of callbacks for w and b-sounding names are the same.
Procedure using bootstrap and p-value for difference <= 0.
p-value = 0.0
H0 is rejected.
Procedure using bootstrap and p-value for difference <= 0.018.
p-value = 0.0384
H0 is rejected.

How big the difference could be?
Procedure using bootstrap and p-value for difference >= 0.045.
p-value = 0.0506
H0 is rejected.


4-) Write a story describing the statistical significance in the context or the original problem.  
The empirical difference between the rates of callbacks for w and b-sounding names is 0.032. In order to know if that difference is in fact statistically significant, I made two tests using bootstrap. The first test had the following null hypothesis: callbacks for w and b-sounding names come from the same distribution. Using permutation, I could observe that the probability of getting a difference equal or higher than the empirical difference under the null hypothesis was very small, so I could reject that hypothesis. The second test had the following null hypothesis: the rates of callbacks for w and b-sounding names are the same. That is a less restrictive hypothesis because I am not saying that callbacks are coming from the same distribution. By comparing 95% confidence intervals for each rate, I could observe that those intervals don't overlap and so I could reject the null hypothesis. Moreover, by also simulating the chance of getting a difference equal or lower than 0, I could observe a very small probability that allowed me to reject the null hypothesis. As a matter of fact, even the probability of getting a difference equal or smaller than 0.018 is small enough to reject the null hypothesis. Therefore, there is a significant difference higher than zero between the rates of callbacks for w and b-sounding names, although that difference is most likely below 0.045. I don't have enough knowledge to say how big a difference must be to indicate racial bias. There is a difference suggesting racial bias nonetheless.

5-) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?  
Here I am only analyzing the possible effect of w and b-sounding names in callback success. Other factors may also play a role in determining the callback success or altering how the names affect it. Without taking those other factors into account (using multiple regression, for example), I cannot say which factor is more important.